# Bayesian Analysis on COVID-19 Antibody Test Results

A simple workbook to determine the probability of an antibody test result matching the truth

In [1]:
import pandas as pd

Bayes's Theorem allows one to calculate a conditional probability based on the information available:

$$P(A|B)=\frac{P(B|A)P(A)}{P(B)}$$

In the context of COVID-19 antibody tests you can use it to calculate the probabilities of the four classifications the antibody test can generate:

Test Result|COVID-19|Not COVID-19
:---:|:---:|:---:
Test + | Tests Positive and has COVID-19 | Tests Positive and Doesn't have COVID-19
Test - | Tests Negative and has COVID-19 | Tests Negative and Doesn't have COVID-19

These correspond to the following Bayesian probabilities:
* $P(\textrm{COVID-19 } | \textrm{ Test } +)$
* $P(\textrm{COVID-19 } | \textrm{ Test } -)$
* $P(\textrm{No COVID-19 } | \textrm{ Test } +)$
* $P(\textrm{No COVID-19 } | \textrm{ Test } -)$

In [2]:
def bayes_theorem(p_a, p_b, p_ba):
    return round((p_ba * p_a) / p_b, 5)

This function will calculate the general population's expected distribution of COVID-19 given the prevalence.
It also will calculate the expected number of true positive, true negative, false positive and false negative results expected for a given test.

In [3]:
columns = ['COVID-19', 'Not COVID-19', 'Total']
index = ['Test +', 'Test -', 'Total']

def case_generator(p, sens, spec, n=10000, columns=columns, index=index):
    '''Takes prevalence (p - decimal) and the antibody test's sensitivity 
    (true positive detection rate; sens - decimal) and specificity (true 
    negative detection rate; spec - decimal) and generates a sample population
    of n. Returns a dataframe mostly for style'''
    
    # determine number of COVID-19 positive and negative
    cvp = round(n * p)
    cvn = round(n * (1 - p))
    
    # determine expected test results of COVID-19 +
    tp = round(sens * cvp)
    fn = round((1 - sens) * cvp)
    
    # determine expected test results of COVID-19 -
    fp = round((1 - spec) * cvn)
    tn = round(spec * cvn)
    
    # sum test results by result
    tot_p = tp + fp
    tot_n = tn + fn
    
    return pd.DataFrame(
        [[tp, fp, tot_p],
         [fn, tn, tot_n],
         [cvp, cvn, n]],
        columns=columns,
        index=index
    )

def probability_calc(p, sens, spec):
    '''Determines the Bayesian probabilities for having COVID-19 and a 
    Positive Test, and having COVID-19 and Negative Test'''
    
    # Sick with positive test, sick with negative test
    p_cp = bayes_theorem(p,
                      sens * p  + (1 - spec),
                      sens
                     )
    p_cn = bayes_theorem(p,
                      ((1 - p) * spec + (1 - sens) * p),
                      (1 - sens) * (1 - p)
                     )
    # True Positive, True Negative
    print(f'True Positive: P(COVID-19 | Test Positive) = {p_cp}')
    print(f'True Negative: P(No COVID-19 | Test Negative) = {1 - p_cn}')
    # False Negative, False Positive
    print(f'False Negative: P(COVID-19 | Test Negative) = {p_cn}')
    print(f'False Positive: P(No COVID-19 | Test Positive) = {1 - p_cp}')
    
    return None

## Exploring Results in Specific States

### Indiana
For [Indiana](https://www.reddit.com/r/science/comments/hdy7yc/a_new_study_from_penn_state_estimates_that_the/fvpil39?utm_source=share&utm_medium=web2x) on 6/23, the estimated percent of infected was 5.6% 95% CL(3.1, 8.3) from [covid-19-projections.com](https://covid19-projections.com/us-in).

For the Antibody test, I used the one my friend was taking since he did the research to see which had the best specificity and sensitivity:

Antibody | Performance Measure | Estimate of Performance | 95% Confidence Interval
:---:|:---:|:---:|:---:
Pan-Ig | Sensitivity (PPA) | 100% (49/49) | (92.7%; 100%)
Pan-Ig | Specificity (NPA) | 100% (400/400) | (99.0%; 100%)
Pan-Ig | PPV at prevalence = 5% | 100% | (83.0.%; 100%)
Pan-Ig | NPV at prevalence = 5% | 100% |  (99.6%; 100%)

#### At 3.1% prevalence:

In [4]:
case_generator(0.031, 0.927, 0.99)

,COVID-19,Not COVID-19,Total
Test +,287,97,384
Test -,23,9593,9616
Total,310,9690,10000


In [5]:
probability_calc(0.031, 0.927, 0.99)

True Positive: P(COVID-19 | Test Positive) = 0.74185
True Negative: P(No COVID-19 | Test Negative) = 0.99772
False Negative: P(COVID-19 | Test Negative) = 0.00228
False Positive: P(No COVID-19 | Test Positive) = 0.25815


#### At 5.6% prevalence:

In [6]:
case_generator(0.056, 0.927, 0.99)

,COVID-19,Not COVID-19,Total
Test +,519,94,613
Test -,41,9346,9387
Total,560,9440,10000


In [7]:
probability_calc(0.056, 0.927, 0.99)

True Positive: P(COVID-19 | Test Positive) = 0.83848
True Negative: P(No COVID-19 | Test Negative) = 0.99589
False Negative: P(COVID-19 | Test Negative) = 0.00411
False Positive: P(No COVID-19 | Test Positive) = 0.16152


#### At 8.3% prevalence:

In [8]:
case_generator(0.083, 0.927, 0.99)

,COVID-19,Not COVID-19,Total
Test +,769,92,861
Test -,61,9078,9139
Total,830,9170,10000


In [9]:
probability_calc(0.083, 0.927, 0.99)

True Positive: P(COVID-19 | Test Positive) = 0.88498
True Negative: P(No COVID-19 | Test Negative) = 0.99392
False Negative: P(COVID-19 | Test Negative) = 0.00608
False Positive: P(No COVID-19 | Test Positive) = 0.11502000000000001


### NJ

[NJ predictions](https://covid19-projections.com/us-nj) are 16.0% 95% CL(10.4%, 23.1%)

#### At 10.4% prevalence:

In [10]:
case_generator(0.104, 0.927, 0.99)

,COVID-19,Not COVID-19,Total
Test +,964,90,1054
Test -,76,8870,8946
Total,1040,8960,10000


In [11]:
probability_calc(0.104, 0.927, 0.99)

True Positive: P(COVID-19 | Test Positive) = 0.90602
True Negative: P(No COVID-19 | Test Negative) = 0.9924
False Negative: P(COVID-19 | Test Negative) = 0.0076
False Positive: P(No COVID-19 | Test Positive) = 0.09397999999999995


#### At 16.0% prevalence:

In [12]:
case_generator(0.16, 0.927, 0.99)

,COVID-19,Not COVID-19,Total
Test +,1483,84,1567
Test -,117,8316,8433
Total,1600,8400,10000


In [13]:
probability_calc(0.16, 0.927, 0.99)

True Positive: P(COVID-19 | Test Positive) = 0.93684
True Negative: P(No COVID-19 | Test Negative) = 0.98837
False Negative: P(COVID-19 | Test Negative) = 0.01163
False Positive: P(No COVID-19 | Test Positive) = 0.06316


#### At 23.1% Prevalence:

In [14]:
case_generator(0.231, 0.927, 0.99)

,COVID-19,Not COVID-19,Total
Test +,2141,77,2218
Test -,169,7613,7782
Total,2310,7690,10000


In [15]:
probability_calc(0.231, 0.927, 0.99)

True Positive: P(COVID-19 | Test Positive) = 0.95538
True Negative: P(No COVID-19 | Test Negative) = 0.98334
False Negative: P(COVID-19 | Test Negative) = 0.01666
False Positive: P(No COVID-19 | Test Positive) = 0.04461999999999999


### IL

[IL predictions](https://covid19-projections.com/us-il) are 7.7% 95% CL(5.1%, 10.5%)

These examples done with [Roche's test](https://diagnostics.roche.com/us/en/roche-blog/key-role-of-specificity-in-covid-19-antibody-test-accuracy.html): sensitivity of 100%; specificity of 99.81%. *Note*: these values have come under fire recently and [independent tests found significantly lower accuracies](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/891598/Evaluation_of_Roche_Elecsys_anti_SARS_CoV_2_PHE_200610_v8.1_FINAL.pdf) (sensitivity of 87%; specificity of 100%). We'll use the lower values to err on the side of caution.

#### At 5.1% prevalence:

In [16]:
case_generator(0.051, 0.87, 1.0)

,COVID-19,Not COVID-19,Total
Test +,444,0,444
Test -,66,9490,9556
Total,510,9490,10000


In [17]:
probability_calc(0.051, 0.87, 1.0)

True Positive: P(COVID-19 | Test Positive) = 1.0
True Negative: P(No COVID-19 | Test Negative) = 0.99342
False Negative: P(COVID-19 | Test Negative) = 0.00658
False Positive: P(No COVID-19 | Test Positive) = 0.0


#### At 7.7% prevalence:

In [18]:
case_generator(0.077, 0.87, 1.0)

,COVID-19,Not COVID-19,Total
Test +,670,0,670
Test -,100,9230,9330
Total,770,9230,10000


In [19]:
probability_calc(0.077, 0.87, 1.0)

True Positive: P(COVID-19 | Test Positive) = 1.0
True Negative: P(No COVID-19 | Test Negative) = 0.9901
False Negative: P(COVID-19 | Test Negative) = 0.0099
False Positive: P(No COVID-19 | Test Positive) = 0.0


#### At 10.5% Prevalence:

In [20]:
case_generator(0.105, 0.87, 1.0)

,COVID-19,Not COVID-19,Total
Test +,914,0,914
Test -,136,8950,9086
Total,1050,8950,10000


In [21]:
probability_calc(0.105, 0.87, 1.0)

True Positive: P(COVID-19 | Test Positive) = 1.0
True Negative: P(No COVID-19 | Test Negative) = 0.98656
False Negative: P(COVID-19 | Test Negative) = 0.01344
False Positive: P(No COVID-19 | Test Positive) = 0.0


## Validation

Tested against examples from [Wayne W. LaMorte, MD, PhD, MPH, Boston University School of Public Health](https://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/BS704_Probability/BS704_Probability6.html#:~:text=Bayes%2C%20who%20was%20a%20reverend,the%20prevalence%20in%20the%20population) and [Coronavirus Antibody Tests Have a Mathematical Pitfall](https://www.scientificamerican.com/article/coronavirus-antibody-tests-have-a-mathematical-pitfall/).

In [22]:
case_generator(0.01,0.99,0.99)

,COVID-19,Not COVID-19,Total
Test +,99,99,198
Test -,1,9801,9802
Total,100,9900,10000


In [23]:
probability_calc(0.01, 0.99, 0.99)

True Positive: P(COVID-19 | Test Positive) = 0.49749
True Negative: P(No COVID-19 | Test Negative) = 0.9999
False Negative: P(COVID-19 | Test Negative) = 0.0001
False Positive: P(No COVID-19 | Test Positive) = 0.50251


In [24]:
case_generator(0.002, 0.85, 0.92154309, n=100000)

,COVID-19,Not COVID-19,Total
Test +,170,7830,8000
Test -,30,91970,92000
Total,200,99800,100000


In [25]:
probability_calc(0.002, 0.85, 0.92154309)

True Positive: P(COVID-19 | Test Positive) = 0.02121
True Negative: P(No COVID-19 | Test Negative) = 0.99967
False Negative: P(COVID-19 | Test Negative) = 0.00033
False Positive: P(No COVID-19 | Test Positive) = 0.97879


In [26]:
case_generator(0.05, 0.95, 0.95, n = 500)

,COVID-19,Not COVID-19,Total
Test +,24,24,48
Test -,1,451,452
Total,25,475,500


In [27]:
probability_calc(0.05, 0.95, 0.95)

True Positive: P(COVID-19 | Test Positive) = 0.48718
True Negative: P(No COVID-19 | Test Negative) = 0.99738
False Negative: P(COVID-19 | Test Negative) = 0.00262
False Positive: P(No COVID-19 | Test Positive) = 0.51282


In [28]:
case_generator(0.25, 0.95, 0.95, n = 500)

,COVID-19,Not COVID-19,Total
Test +,119,19,138
Test -,6,356,362
Total,125,375,500


In [29]:
probability_calc(0.25, 0.95, 0.95)

True Positive: P(COVID-19 | Test Positive) = 0.82609
True Negative: P(No COVID-19 | Test Negative) = 0.98707
False Negative: P(COVID-19 | Test Negative) = 0.01293
False Positive: P(No COVID-19 | Test Positive) = 0.17391
